# Source Valid Data

In [1]:
import pandas as pd
import numpy as np
import math
from statistics import mean 


In [2]:
df = pd.read_csv('../../../datasets/dr-nubel/strain_stress/AGR-194.csv')
# df = pd.read_csv('../../../datasets/dr-nubel/strain_stress/AGR-180.csv')
df.head(2)

,strain,stress
0,-0.0149,0.428
1,0.0003,1.925


To calculate the shear strength of the shale samples using the data you have, you can follow these steps:

-    1- Compute the mean and standard deviation of the triaxial test data for each of the 15 shale samples.
-    2- Compute the confining pressure for each sample by averaging the values of the three confining pressures used in the triaxial test.
-    3- Calculate the deviatoric stress (σ1 - σ3) for each sample, where σ1 is the maximum stress applied during the triaxial test and σ3 is the confining pressure.
-    4- Calculate the mean stress (σ1 + 2σ3)/3 for each sample.
-    5- Calculate the angle of internal friction (φ) using the formula:

***tan(φ) = (σ1 - σ3)/(2P)***
<p style="color:red">tan(φ) = (σ1 - σ3)/(2P)</p>
where P is the mean stress.
Calculate the cohesion (c) using the formula:

***c = (τ1 + τ2 + τ3)/3 - σn tan(φ)***
<p style="color:red">c = (τ1 + τ2 + τ3)/3 - σn tan(φ)</p>

where τ1, τ2, and τ3 are the shear stresses at failure for the three stress paths tested in the triaxial test, and σn is the normal stress.
Calculate the shear strength (τ) using the formula:

***τ = c + σn tan(φ)***
<p style="color:red">τ = c + σn tan(φ)</p>

where σn is the mean stress.
By following these steps, you can calculate the shear strength of the shale samples from the triaxial test data. Once you have the shear strength values, you can use them as the target variable for training an artificial neural network to predict the shear strength of other shale samples based on their triaxial and cone penetration test data.

### Manual Calculations

#### angle - tan(φ) = (σ1 - σ3)/(2P)
- σ1 = 368.506
- σ3 = 200
- P = mean stress

In [3]:
q1 = df.stress.max()
q3 = 200
p = df.stress.mean()

q1,q3,p

(336.688, 200, 210.69874193548384)

In [4]:
tan_tita = (q1 - q3)/(2*p)
tan_tita # slope

0.32436833448643465

In [5]:
# tita = 1/math.tan(tan_tita)
tita = np.degrees(tan_tita)
tita

18.584936573760498

In [6]:
# tan_tita = np.radians(tita)
# tan_tita

In [7]:
# normal ang
qm = (q1 + q3)/2
tm = (q1 - q3)/2

print(qm,tm)

q = qm - (tm * math.sin(tita))
print(q)

268.344 68.344
286.21881855966274


In [8]:
# effective stress
eq = q - q3
eq

86.21881855966274

Effective stress = normal stress - pore water pressure  (σ = σ1 - σ3)

#### Intercept - c = (τ1 + τ2 + τ3)/3 - σn tan(φ)
- τ1 = 368.506
- τ2 = 721.295
- τ3 = 705.545

In [9]:
max_stress = max(df.stress) # find the maximum stress value
print(df[df.stress == max_stress].index[0])
t2 = df[df.stress == max_stress].index[0]
# max_strain = df[df.stress == max_stress].values[0][0]
max_stress1 = df.stress[t2-2] # find the corresponding strain value
max_stress2 = df.stress[t2-1]
max_stress3 = df.stress[t2]
max_stress1,max_stress2,max_stress3

334


(336.486, 336.588, 336.688)

In [10]:
c = (max_stress1+max_stress3+max_stress3)/3 - (q*tan_tita)
print(c)

s,c1=np.polyfit(df.strain,df.stress,1)
print(c1)

243.7803451917938
146.08789040385088


In [11]:
slope = np.diff(df.stress) / np.diff(df.strain)
slope
peak_index = np.argmax(slope)
print(peak_index)
# Calculate cohesion
cohesion = (slope[peak_index-1] + slope[peak_index+1]) / 2

print("Cohesion:", cohesion)

5
Cohesion: 5882.4999999999945


In [12]:
print(c)
print(c1)
print(cohesion)

243.7803451917938
146.08789040385088
5882.4999999999945


#### Tau - τ = c + σn tan(φ)

In [13]:
t = c + (eq*tan_tita)
print(t)
t = c1 + (eq*tan_tita)
print(t)
t = cohesion + (eq*tan_tita)

print(t)

271.7469997693797
174.05454498143678
5910.466654577581
